In [36]:
!pip install --upgrade tables
!pip install eli5
!pip install xgboost

Requirement already up-to-date: tables in /usr/local/lib/python3.6/dist-packages (3.6.1)


In [0]:
import pandas as pd
import numpy as np

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

import xgboost as xgb

from sklearn.metrics import mean_absolute_error as mea 
from sklearn.model_selection import cross_val_score, KFold

import eli5
from eli5.sklearn import PermutationImportance

In [38]:
cd /content/drive/My Drive/Colab Notebooks/Matrix_repo/m_2/Car-Price-Prediction/data

/content/drive/My Drive/Colab Notebooks/Matrix_repo/m_2/Car-Price-Prediction/data


In [81]:
df = pd.read_hdf('car.h5')
df.shape

(106494, 155)

In [0]:
SUFFIX_CAT = '__cat'
for feat in df.columns: 
  if isinstance(df[feat][0], list):continue

  factorized_values = df[feat].factorize()[0]
  if SUFFIX_CAT in feat:
    df[feat] = factorized_values
  else:
    df[feat + SUFFIX_CAT] = factorized_values

In [88]:
cat_feats = [x for x in df.columns if SUFFIX_CAT in x]
cat_feats = [x for x in cat_feats if 'price' not in x]
len(cat_feats)


151

In [0]:
X= df[cat_feats].values
y = df['price_value'].values

In [0]:
def run_model(model,feats):
  X= df[feats].values
  y = df['price_value'].values

  scores = cross_val_score(model, X,y, cv=3, scoring = 'neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

decision tree


In [90]:
run_model(DecisionTreeRegressor(max_depth=5), cat_feats)

(-19785.44736156864, 62.60560963378086)

random forest


In [91]:
model = RandomForestRegressor(max_depth=5, n_estimators=50, random_state=0)
run_model(model, cat_feats)

(-18749.47797823352, 38.63721077745084)

XGBoost

In [46]:
xgb_params={
    'max_depth': 5,
    'n_estimators': 50,
    'learning_rate': 0.1,
    'seed':0
}

run_model(xgb.XGBRegressor(**xgb_params),cat_feats)

[18:48:29] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:48:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:49:07] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13108.379065811214, 74.32158265003798)

In [47]:
m=xgb.XGBRegressor(max_depth=5, n_estimators=50,learning_rate=0.1, seed=0)
m.fit(X,y)

imp = PermutationImportance(m, random_state=0).fit(X,y)
eli5.show_weights(m,feature_names=cat_feats)

[18:49:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Weight,Feature
0.0951,param_faktura-vat__cat
0.0949,feature_kamera-cofania__cat
0.0669,feature_łopatki-zmiany-biegów__cat
0.0664,param_napęd__cat
0.0560,param_skrzynia-biegów__cat
0.0423,feature_asystent-pasa-ruchu__cat
0.0411,param_stan__cat
0.0329,feature_światła-led__cat
0.0279,feature_bluetooth__cat
0.0263,feature_regulowane-zawieszenie__cat


In [0]:
feats = ["param_faktura-vat__cat",
"feature_kamera-cofania__cat",
"feature_łopatki-zmiany-biegów__cat",
"param_napęd__cat",
"param_skrzynia-biegów__cat",
"feature_asystent-pasa-ruchu__cat",
"param_stan__cat",
"feature_światła-led__cat",
"feature_bluetooth__cat",
"feature_regulowane-zawieszenie__cat",
"feature_wspomaganie-kierownicy__cat",
"feature_system-start-stop__cat",
"feature_światła-do-jazdy-dziennej__cat",
"feature_światła-xenonowe__cat",
"feature_czujniki-parkowania-przednie__cat",
"param_moc__cat",
"param_rok-produkcji__cat",
"param_pojemność-skokowa__cat",
"feature_asystent-parkowania__cat",
"seller_name__cat"]

In [72]:
run_model(xgb.XGBRegressor(**xgb_params), feats)

[19:51:41] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:51:45] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:51:49] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-14244.091132108233, 147.67804126549066)

In [74]:
df['param_rok-produkcji']=df['param_rok-produkcji'].map(lambda x: -1 if str(x)== 'None' else int(x))

feats = ["param_faktura-vat__cat",
"feature_kamera-cofania__cat",
"feature_łopatki-zmiany-biegów__cat",
"param_napęd__cat",
"param_skrzynia-biegów__cat",
"feature_asystent-pasa-ruchu__cat",
"param_stan__cat",
"feature_światła-led__cat",
"feature_bluetooth__cat",
"feature_regulowane-zawieszenie__cat",
"feature_wspomaganie-kierownicy__cat",
"feature_system-start-stop__cat",
"feature_światła-do-jazdy-dziennej__cat",
"feature_światła-xenonowe__cat",
"feature_czujniki-parkowania-przednie__cat",
"param_moc__cat",
"param_rok-produkcji",
"param_pojemność-skokowa__cat",
"feature_asystent-parkowania__cat",
"seller_name__cat"]

run_model(xgb.XGBRegressor(**xgb_params), feats)

[19:52:41] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:52:45] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:52:49] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-12606.589949595165, 70.30901375926933)

In [92]:
df['param_moc'] = df['param_moc'].map(lambda x: -1 if str(x) == 'None' else int(str(x).split(' ')[0]) )
df['param_rok-produkcji']=df['param_rok-produkcji'].map(lambda x: -1 if str(x)== 'None' else int(x))


feats = ["param_faktura-vat__cat",
"feature_kamera-cofania__cat",
"feature_łopatki-zmiany-biegów__cat",
"param_napęd__cat",
"param_skrzynia-biegów__cat",
"feature_asystent-pasa-ruchu__cat",
"param_stan__cat",
"feature_światła-led__cat",
"feature_bluetooth__cat",
"feature_regulowane-zawieszenie__cat",
"feature_wspomaganie-kierownicy__cat",
"feature_system-start-stop__cat",
"feature_światła-do-jazdy-dziennej__cat",
"feature_światła-xenonowe__cat",
"feature_czujniki-parkowania-przednie__cat",
"param_moc",
"param_rok-produkcji",
"param_pojemność-skokowa__cat",
"feature_asystent-parkowania__cat",
"seller_name__cat"]

run_model(xgb.XGBRegressor(**xgb_params), feats)

[20:06:41] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:06:45] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:06:49] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-10666.284949149864, 90.31134736896897)

In [93]:
df['param_pojemność-skokowa']=df['param_pojemność-skokowa'].map(lambda x: -1 if str(x) == 'None' else int(str(x).split('cm')[0].replace(' ','')) )

feats = ["param_faktura-vat__cat",
"feature_kamera-cofania__cat",
"feature_łopatki-zmiany-biegów__cat",
"param_napęd__cat",
"param_skrzynia-biegów__cat",
"feature_asystent-pasa-ruchu__cat",
"param_stan__cat",
"feature_światła-led__cat",
"feature_bluetooth__cat",
"feature_regulowane-zawieszenie__cat",
"feature_wspomaganie-kierownicy__cat",
"feature_system-start-stop__cat",
"feature_światła-do-jazdy-dziennej__cat",
"feature_światła-xenonowe__cat",
"feature_czujniki-parkowania-przednie__cat",
"param_moc",
"param_rok-produkcji",
"param_pojemność-skokowa",
"feature_asystent-parkowania__cat",
"seller_name__cat"]

run_model(xgb.XGBRegressor(**xgb_params), feats)

[20:06:57] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:07:01] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:07:05] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-10418.18729276653, 72.80569459652894)